### Summary:

This notebook contains our standard code for merging the Mouse PDAC samples into one Seurat object and removing doublets using Scrublet. This notebook also contains basic QC metrics. 

In [1]:
# Always run this first!!
#essential reticulate functions that allow us to use python packages in R
#you'll need a conda env with 'reticulate', leidenalg' and 'pandas' installed to do this
#then route reticulate to the python installed in that conda env with the below functions
#change the path to whichever conda env you installed reticulate in
Sys.setenv(RETICULATE_PYTHON="/home/yuna/.conda/envs/reticulate/bin/python")
library(reticulate)
reticulate::use_python("/home/yuna/.conda/envs/reticulate/bin/python")
reticulate::use_condaenv("/home/yuna/.conda/envs/reticulate")
reticulate::py_module_available(module='leidenalg') #needs to be TRUE
reticulate::import('leidenalg') #good to make sure this doesn't error

[1] TRUE

Module(leidenalg)

In [2]:
options(warn=-1)
suppressMessages(library(hdf5r))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(EnsDb.Mmusculus.v79))
suppressMessages(library(dplyr))
suppressMessages(library(ggplot2))
suppressMessages(library(Matrix))
suppressMessages(library(harmony))
suppressMessages(library(data.table))
suppressMessages(library(ggpubr))
suppressMessages(library(plyr))
suppressMessages(library(sparseMatrixStats))
suppressMessages(library(stringr))

# Read in individually processed data and merge
## RNA matrices

In [77]:
samples <- c('KA_41_1_2_KA_39_1_2','KA_42_1_2_KA_40_1_2')

In [4]:
# Read in the RDS files and wipe out the ATAC and SCT components
adatas <- list()
for (sample in samples){
    adata <- readRDS(file = sprintf("/nfs/lab/projects/mouse_pdac/processing/pipeline_outputs/%s/final_filtered_genotypes.rds",sample))
    adatas[[sample]] <- adata
    DefaultAssay(adatas[[sample]]) <- "RNA"
    adatas[[sample]][['ATAC']] <- NULL
    adatas[[sample]][['SCT']] <- NULL
}

In [5]:
# Merge RNA objects (make sure you add sample prefix to these or it won't combine with ATAC)
adata <- merge(adatas[[samples[[1]]]], y=adatas[samples[2:length(samples)]], add.cell.ids=samples, project='mouse_pdac')
adata$library <- sapply(strsplit(rownames(adata[[]]), '_'), '[[', 2)
adatas <- NULL

## ATAC matrices

In [9]:
# Make the merged fragment file (update all filepaths and variables!)
system(sprintf('for SAMPLE in %s; do zcat /nfs/lab/projects/mouse_pdac/data/$SAMPLE/outs/atac_fragments.tsv.gz | awk -v SAMPLE=$SAMPLE \'BEGIN{FS=OFS="\t"} {print $1,$2,$3,SAMPLE"_"$4,$5}\'; done | sort -k1,1 -k2,2n -S 64G | bgzip -c -@ 16 > /nfs/lab/projects/mouse_pdac/2samples_merged.atac_fragments.tsv.gz', paste0(samples, collapse=' ')))
system('tabix -p bed /nfs/lab/projects/mouse_pdac/2samples_merged.atac_fragments.tsv.gz')
outdir <- "/nfs/lab/ylee/mouse_pdac/"
frag.file <- file.path(outdir,'2samples_merged.atac_fragments.tsv.gz')

In [7]:
# Read in ATAC data from the lfm matrices (sm workaround method for now)
# Load in starting ATAC long format matrices to a list
atacs <- list()
for (sample in samples) {
    wd <- sprintf('/nfs/lab/projects/mouse_pdac/processing/pipeline_outputs/%s/', sample)
    atacs[[sample]] <- read.table(file.path(wd, 'atac.long_fmt.filtered_barcode.mtx.gz'), sep='\t', header=FALSE, stringsAsFactors=FALSE)
    atacs[[sample]]$V1 <- as.factor(atacs[[sample]]$V1)
    atacs[[sample]]$V2 <- as.factor(paste0(sample, '_', atacs[[sample]]$V2))
}

In [8]:
# Make the set of ALL windows from all samples
windows <- c()
for (sample in samples) {
    print(paste(sample,Sys.time(),sep=': '))
    # Make a temp sparse matrix (can't be final bc doesn't have all windows)
    temp_sm <- with(atacs[[sample]],sparseMatrix(i=as.numeric(V1), j=as.numeric(V2), x=V3, dimnames=list(levels(V1), levels(V2))))
    # Add to the common windows set
    windows <- union(windows,row.names(temp_sm))
    }
rm(temp_sm) # Deleting for memory reasons
windows <- sort(windows) 

[1] "KA_41_1_2_KA_39_1_2: 2024-01-09 11:01:28"
[1] "KA_42_1_2_KA_40_1_2: 2024-01-09 11:01:51"


In [9]:
#function which takes in a list of long format atac_fragment dfs with 
#sample names (df), an overall windows file (windows) and then makes these 
#into sparse matrices and merges them together

#modified to take in the hvws set and use those... will still check if 
#there's any missing windows and add those, so only a few changes!

merge_sparse_matrices_hvws <- function(dfs, windows){
    samples <- names(dfs)
    for (sample in samples){
        #get missing windows list for this sample
        print(paste(sample,Sys.time(),sep=': '))
        df <- dfs[[sample]]
        mis_windows <- windows[!windows %in% levels(df$V1)]
        
        #add in any missing windows as zero counts for one barcode
        if (length(mis_windows) > 0){
            print('Adding missing windows')
            #create a new long format matrix with the missing windows added as 0 counts
            filler_bc <- as.character(df$V2[[1]])
            print(paste("Using filler BC:",filler_bc,sep=" "))
            new_rows <- cbind(as.data.frame(mis_windows),
                              as.data.frame(rep(filler_bc),length(mis_windows)),
                              as.data.frame(rep(0,length(mis_windows))))
            colnames(new_rows) <- c("V1","V2","V3")
            lfm <- rbind(df,new_rows)
        #if there aren't any missing windows, set lfm to df
        } else {
            print('No windows were missing')
            lfm <- df
        }
        
        #cut down lfm to just be the hvws (windows)
        lfm_cut <- lfm[lfm$V1 %in% windows,]
        
        #set the levels of the lfm based on the desired bc order and reorder V1
        lfm_cut$V1 <- factor(lfm_cut$V1, levels=windows)
        lfm2 <- lfm_cut[order(lfm_cut$V1),]
        lfm2$V2 <- as.factor(lfm2$V2)
        
        if (sample == samples[1]){
            #if first sample, will make the overall sparse matrix 
            overall_sm <- with(lfm2,sparseMatrix(i=as.numeric(V1), j=as.numeric(V2), x=V3, dimnames=list(levels(V1), levels(V2))))
            
        } else {
            #otherwise, convert into a sparse matrix and add to the overall one
            sm = with(lfm2,sparseMatrix(i=as.numeric(V1), j=as.numeric(V2), x=V3, dimnames=list(levels(V1), levels(V2))))
            overall_sm = cbind(overall_sm, sm) 
        }
    }
    return(overall_sm)
}

In [10]:
overall_sm2 <- merge_sparse_matrices_hvws(atacs,windows)

[1] "KA_41_1_2_KA_39_1_2: 2024-01-09 11:02:07"
[1] "Adding missing windows"
[1] "Using filler BC: KA_41_1_2_KA_39_1_2_AACATTGTCAAGGACA-1"
[1] "KA_42_1_2_KA_40_1_2: 2024-01-09 11:04:47"
[1] "Adding missing windows"
[1] "Using filler BC: KA_42_1_2_KA_40_1_2_AAGGCCCTCAAACCTA-1"


In [11]:
#check if the windows of overall_sm are sorted
sorted_windows2 <- sort(row.names(overall_sm2))

In [12]:
#continue to make Seurat compatible object
atac <- overall_sm2
suppressMessages(annotations <- GetGRangesFromEnsDb(ensdb=EnsDb.Mmusculus.v79))
seqlevelsStyle(annotations) <- 'UCSC'
genome(annotations) <- 'mm10'

In [13]:
#make sure this is the correct merged fragment file!!
frag.file <- '/nfs/lab/projects/mouse_pdac/temp.tsv.gz'
atac_assay <- CreateChromatinAssay(counts=atac, sep=c('-', '-'), genome='mm10', fragments=frag.file, min.cells=10, min.features=-1, annotation=annotations)

Computing hash



In [14]:
suppressMessages(adata[['ATAC']] <- atac_assay)

## Add in sample metadata from `per_barcode_metrics.csv`

In [15]:
# add qc metrics from 10x
qcs <- list()
for (sample in samples) {
    wd <- sprintf('/nfs/lab/projects/mouse_pdac/data/%s/outs', sample)
    qc <- read.table(file.path(wd, 'per_barcode_metrics.csv'), sep=',', header=TRUE, stringsAsFactors=1)
    qc <- qc[qc$is_cell==1,]
    qc$gex_barcode <- paste0(sample, '_', qc$gex_barcode)
    qcs[[sample]] <- qc
}
qc <- as.data.frame(rbindlist(qcs))
rownames(qc) <- qc$gex_barcode
qc <- qc[Cells(adata), 6:length(colnames(qc))]
adata <- AddMetaData(adata, qc)
qc <- qcs <- NULL

# Perform analyses and clustering

In [16]:
# RNA analysis
DefaultAssay(adata) <- 'RNA'
suppressWarnings(adata <- SCTransform(adata, verbose=FALSE))
adata <- RunPCA(adata)
adata <- RunHarmony(adata, group.by.vars='library', assay.use='SCT', reduction.save='harmony.rna')
adata <- RunUMAP(adata, dims=1:50, reduction='harmony.rna', reduction.name='umap.rna', reduction.key='rnaUMAP_')

# ATAC analysis
# We exclude the first dimension as this is typically correlated with sequencing depth
DefaultAssay(adata) <- 'ATAC'
adata <- RunTFIDF(adata)
adata <- FindTopFeatures(adata, min.cutoff='q0', verbose=FALSE)
adata <- RunSVD(adata)
hm_atac <- HarmonyMatrix(Embeddings(adata, reduction='lsi'), adata$library, do_pca=FALSE)
adata[['harmony.atac']] <- CreateDimReducObject(embeddings=hm_atac, key='atac_', assay='ATAC')
adata <- RunUMAP(adata, dims=2:50, reduction='harmony.atac', reduction.name='umap.atac', reduction.key='atacUMAP_')

PC_ 1 
Positive:  Arhgap15, Bank1, Dock2, Aff3, Ripor2, Inpp4b, Bach2, Ptprc, Ccnd3, Pax5 
	   Btla, Skap1, Dock10, Ikzf1, Elmo1, Prkcb, Ikzf3, Satb1, Papss2, Gm2682 
	   March1, Dpyd, Mrc1, Grap2, Wdfy4, Sis, Ptpn22, Immp2l, Mgam, Kcnq5 
Negative:  Slit3, Col1a2, Col3a1, Col5a2, Pdzrn3, Kcnma1, Prickle1, Kif26b, Bnc2, Epha3 
	   Tnc, Ror1, Sox5, Lhfp, Fbn1, Cacna1c, Col5a1, Col1a1, Cald1, Brinp3 
	   Rbms3, Nhs, Cdh11, Tenm3, Magi2, Prkg1, Meg3, Mast4, Gpc6, C1qtnf7 
PC_ 2 
Positive:  Arhgap15, Bank1, Ebf1, Dock2, Aff3, Ripor2, Inpp4b, Bach2, Ptprc, Zeb2 
	   Ccnd3, Dock10, Skap1, St6galnac3, Pax5, Btla, Plxdc2, Ikzf1, Mrc1, Prkcb 
	   Kcnq5, Slc8a1, Satb1, Mef2c, March1, Fli1, Ikzf3, Gm2682, Wdfy4, Grap2 
Negative:  Dpyd, Papss2, Sis, Mgam, Slc13a1, Abcc2, Clec2h, Mecom, Pcsk5, Nckap5 
	   Tmprss15, Malrd1, Slc6a20a, Sult6b2, Cftr, Dmbt1, Magi1, Agmo, Mme, Apob 
	   Enpp3, Slc5a1, Hnf4g, Cyp3a13, Adh6a, Muc13, Cobl, Plac8, Shroom3, Onecut2 
PC_ 3 
Positive:  Dpyd, Papss2, Sis, Pcsk5,

In [17]:
# Combining Modalities
adata <- FindMultiModalNeighbors(adata, reduction.list=list('harmony.rna', 'harmony.atac'), dims.list=list(1:50, 2:50))
adata <- RunUMAP(adata, nn.name='weighted.nn', reduction.name='umap.wnn', reduction.key='wnnUMAP_')
adata <- FindClusters(adata, graph.name='wsnn', algorithm=4,  resolution = 0.25, verbose=FALSE, method = 'igraph')

adata$log_nCount_ATAC=log(adata$nCount_ATAC)
adata$log_nCount_SCT=log(adata$nCount_SCT)
adata$log_nFeature_ATAC=log(adata$nFeature_ATAC)
adata$log_nFeature_SCT=log(adata$nFeature_SCT)

Calculating cell-specific modality weights

Finding 20 nearest neighbors for each modality.

Calculating kernel bandwidths

Finding multimodal neighbors

Constructing multimodal KNN graph

Constructing multimodal SNN graph

12:26:03 UMAP embedding parameters a = 0.9922 b = 1.112

12:26:07 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 20

12:26:13 Initializing from normalized Laplacian + noise (using irlba)

12:26:37 Commencing optimization for 200 epochs, with 365442 positive edges

12:26:56 Optimization finished



## Visualize clustering

In [81]:
# Prints out the UMAP for intermediate clustering step, categorized by seurat clusters
options(repr.plot.width=18, repr.plot.height=6)
p1 <- DimPlot(adata, reduction='umap.rna', group.by='seurat_clusters', label=TRUE, label.size=6, repel=TRUE) + ggtitle('RNA')
p1 <- p1 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('RNA only')
p2 <- DimPlot(adata, reduction='umap.atac', group.by='seurat_clusters', label=TRUE, label.size=6, repel=TRUE) + ggtitle('ATAC')
p2 <- p2 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC only')
p3 <- DimPlot(adata, reduction='umap.wnn', group.by='seurat_clusters', label=TRUE, label.size=6, repel=TRUE) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('Combined')
# p1 + p2 + p3 & NoLegend() & theme(plot.title=element_text(hjust=0.5))

# Visualize Scrublet 

In [78]:
# Read in the first set of Scrublet doublets
scrub1 <- read.table("/nfs/lab/projects/mouse_pdac/processing/pipeline_outputs/KA_41_1_2_KA_39_1_2/matrix_market/scrublet_predicted_doublets_cutoff0.25.txt")
scrub41_doublets_set = paste('KA_41_1_2_KA_39_1_2', scrub1[scrub1$V2=='True',1], sep = "_")

# Create vector of assignments with BC names
cells <- Cells(adata)
scrublet.raw_combined = rep('singlet', length(cells))
names(scrublet.raw_combined) <- cells
scrublet.raw_combined[scrub41_doublets_set] <- 'doublet'

# Add metadata and visualize on the combined UMAP
adata <- AddMetaData(adata, scrublet.raw_combined, col.name='scrublet41_0.25.raw_combined')

# # Visualize Scrublet multiplets on the combined UMAP -- uncomment out to plot
# options(repr.plot.width=10, repr.plot.height=10)
# DimPlot(adata, reduction='umap.wnn', group.by = "scrublet41_0.25.raw_combined", pt.size = 1, raster=FALSE,
#         cols = c("singlet" = "grey", "doublet" = "blue"), 
#         order = c("doublet", "singlet"))

In [79]:
# Read in a the first set of Scrublet doublets
scrub1 <- read.table("/nfs/lab/projects/mouse_pdac/processing/pipeline_outputs/KA_42_1_2_KA_40_1_2/matrix_market/scrublet_predicted_doublets_cutoff0.25.txt")
scrub42_doublets_set = paste('KA_42_1_2_KA_40_1_2', scrub1[scrub1$V2=='True',1], sep = "_")

# Create vector of assignments with BC names
cells <- Cells(adata)
scrublet.raw_combined = rep('singlet', length(cells))
names(scrublet.raw_combined) <- cells
scrublet.raw_combined[scrub42_doublets_set] <- 'doublet'

# Add metadata and visualize on the combined UMAP
adata <- AddMetaData(adata, scrublet.raw_combined, col.name='scrublet42_0.25.raw_combined')

# # Visualize Scrublet doublets on the combined UMAP -- uncomment out to plot
# options(repr.plot.width=10, repr.plot.height=10)
# DimPlot(adata, reduction='umap.wnn', group.by = "scrublet42_0.25.raw_combined", pt.size = 1, raster=FALSE,
#         cols = c("singlet" = "grey", "doublet" = "blue"), 
#         order = c("doublet", "singlet"))

In [25]:
# Concatenate the two default and two set doublet lists
scrub_set <- c(scrub41_doublets_set,scrub42_doublets_set)

In [26]:
cells <- Cells(adata)
scrublet.raw_combined_set = rep('singlet', length(cells))
names(scrublet.raw_combined_set) <- cells
scrublet.raw_combined_set[scrub_set] <- 'doublet'

In [27]:
# Add in the conatenated lists to metadata
adata <- AddMetaData(adata, scrublet.raw_combined_set, col.name='scrublet_set')

In [80]:
# # Visualize concatenated Scrublet doublets on the combined UMAP -- uncomment out to plot
# options(repr.plot.width=10, repr.plot.height=10)
# DimPlot(adata, reduction='umap.wnn', group.by = "scrublet_set", pt.size = 1, raster=FALSE,
#         cols = c("singlet" = "grey", "doublet" = "blue"), 
#         order = c("doublet", "singlet"))

# Doublet Removal: Subsetting out adata for Scrublet set doublets

In [29]:
adata_sub <- subset(adata, subset = scrublet_set == 'singlet')

## Perform analyses and clustering

In [30]:
# RNA analysis
DefaultAssay(adata_sub) <- 'RNA'
suppressWarnings(adata_sub <- SCTransform(adata_sub, verbose=FALSE))
adata_sub <- RunPCA(adata_sub)
adata_sub <- RunHarmony(adata_sub, group.by.vars='library', assay.use='SCT', reduction.save='harmony.rna')
adata_sub <- RunUMAP(adata_sub, dims=1:50, reduction='harmony.rna', reduction.name='umap.rna', reduction.key='rnaUMAP_')

# ATAC analysis
# We exclude the first dimension as this is typically correlated with sequencing depth
DefaultAssay(adata_sub) <- 'ATAC'
adata_sub <- RunTFIDF(adata_sub)
adata_sub <- FindTopFeatures(adata_sub, min.cutoff='q0', verbose=FALSE)
adata_sub <- RunSVD(adata_sub)
hm_atac <- HarmonyMatrix(Embeddings(adata_sub, reduction='lsi'), adata_sub$library, do_pca=FALSE)
adata_sub[['harmony.atac']] <- CreateDimReducObject(embeddings=hm_atac, key='atac_', assay='ATAC')
adata_sub <- RunUMAP(adata_sub, dims=2:50, reduction='harmony.atac', reduction.name='umap.atac', reduction.key='atacUMAP_')

PC_ 1 
Positive:  Slit3, Col1a2, Col3a1, Col5a2, Pdzrn3, Kcnma1, Prickle1, Kif26b, Bnc2, Epha3 
	   Ror1, Sox5, Tnc, Lhfp, Rbms3, Fbn1, Cacna1c, Col5a1, Col1a1, Cald1 
	   Tshz2, Mast4, Tenm3, Nhs, Brinp3, Cdh11, Meg3, Prkg1, Gpc6, Magi2 
Negative:  Arhgap15, Bank1, Dock2, Aff3, Ripor2, Inpp4b, Bach2, Ptprc, Dock10, Pax5 
	   Ccnd3, Btla, Skap1, Ikzf1, Prkcb, Ikzf3, March1, Mrc1, Satb1, Elmo1 
	   Gm2682, Wdfy4, Grap2, Kcnq5, Ptpn22, Ighd, Ebf1, Itga4, Lyn, Ankrd44 
PC_ 2 
Positive:  Arhgap15, Ebf1, Bank1, Dock2, Aff3, Slit3, Zeb2, Ripor2, Inpp4b, Bach2 
	   Plxdc2, Col1a2, Ptprc, Dock10, Slc8a1, Col3a1, Ccnd3, Col5a2, St6galnac3, Mrc1 
	   Sox5, Pax5, Btla, Ikzf1, Epha3, Skap1, Prickle1, Prkcb, Bnc2, Arhgap24 
Negative:  Dpyd, Papss2, Sis, Mgam, Slc13a1, Abcc2, Mecom, Clec2h, Pcsk5, Cftr 
	   Nckap5, Dmbt1, Malrd1, Tmprss15, Slc6a20a, Agmo, Magi1, Sult6b2, Hnf4g, Apob 
	   Mme, Slc5a1, Enpp3, Shroom3, Cyp3a13, Onecut2, Cobl, Muc13, Adh6a, Plac8 
PC_ 3 
Positive:  Dmbt1, Magi1, Mecom, 

In [31]:
# Combining Modalities
adata_sub <- FindMultiModalNeighbors(adata_sub, reduction.list=list('harmony.rna', 'harmony.atac'), dims.list=list(1:50, 2:50))
adata_sub <- RunUMAP(adata_sub, nn.name='weighted.nn', reduction.name='umap.wnn', reduction.key='wnnUMAP_')
adata_sub <- FindClusters(adata_sub, graph.name='wsnn', algorithm=4,  resolution = 0.25, verbose=FALSE, method = 'igraph')

adata_sub$log_nCount_ATAC=log(adata_sub$nCount_ATAC)
adata_sub$log_nCount_SCT=log(adata_sub$nCount_SCT)
adata_sub$log_nFeature_ATAC=log(adata_sub$nFeature_ATAC)
adata_sub$log_nFeature_SCT=log(adata_sub$nFeature_SCT)

Calculating cell-specific modality weights

Finding 20 nearest neighbors for each modality.

Calculating kernel bandwidths

Finding multimodal neighbors

Constructing multimodal KNN graph

Constructing multimodal SNN graph

14:59:42 UMAP embedding parameters a = 0.9922 b = 1.112

14:59:44 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 20

14:59:49 Initializing from normalized Laplacian + noise (using irlba)

15:00:12 Commencing optimization for 200 epochs, with 355834 positive edges

15:00:23 Optimization finished



## Visualize clustering

In [82]:
# Prints out the UMAP after removing Scrublet doublets and reclustering
options(repr.plot.width=18, repr.plot.height=6)
p1 <- DimPlot(adata_sub, reduction='umap.rna', group.by='seurat_clusters', label=TRUE, label.size=6, repel=TRUE) + ggtitle('RNA')
p1 <- p1 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('RNA only')
p2 <- DimPlot(adata_sub, reduction='umap.atac', group.by='seurat_clusters', label=TRUE, label.size=6, repel=TRUE) + ggtitle('ATAC')
p2 <- p2 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC only')
p3 <- DimPlot(adata_sub, reduction='umap.wnn', group.by='seurat_clusters', label=TRUE, label.size=6, repel=TRUE) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('Combined')
# p1 + p2 + p3 & NoLegend() & theme(plot.title=element_text(hjust=0.5))

In [83]:
#p1 is the UMAP split by genotype, p2 is a bar plot showing the proportion of genotype per seurat cluster
options(repr.plot.width=27, repr.plot.height=15)
p1 <- DimPlot(adata_sub, reduction='umap.rna', group.by='Genotype', split.by='Genotype', label=FALSE, label.size=6, repel=TRUE)
adata_sub$value <- 1
p2 <- ggplot(adata_sub[[]], aes(fill=library, y=value, x=seurat_clusters)) + geom_bar(position=position_fill(reverse=TRUE), stat='identity') + xlab('') + ylab('percentage') + theme_light()
# p1 / p2

In [84]:
# p1 is a Feature Plot showing different features on the combined UMAP
options(repr.plot.width=20, repr.plot.height=5)
p1 <- FeaturePlot(adata_sub, reduction='umap.rna', features=c('log_nCount_ATAC','log_nFeature_ATAC','log_nCount_SCT','log_nFeature_SCT'), 
                  cols=c('lightgrey', 'red'), ncol=4) + xlab('UMAP 1') + ylab('UMAP 2')
# p1 & theme(plot.title=element_text(hjust=0.5))

In [85]:
# Figure prints out violin pltos showing different features grouped by seurat clusters
options(repr.plot.width=24, repr.plot.height=16)
p1 <- VlnPlot(adata_sub, features='nCount_SCT', group.by='seurat_clusters', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_sub$nCount_SCT), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p2 <- VlnPlot(adata_sub, features='nFeature_SCT', group.by='seurat_clusters', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_sub$nFeature_SCT), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p3 <- VlnPlot(adata_sub, features='nCount_ATAC', group.by='seurat_clusters', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_sub$nCount_ATAC), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p4 <- VlnPlot(adata_sub, features='nFeature_ATAC', group.by='seurat_clusters', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_sub$nFeature_ATAC), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p5 <- VlnPlot(adata_sub, features='nCount_RNA', group.by='seurat_clusters', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_sub$nCount_SCT), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p6 <- VlnPlot(adata_sub, features='nFeature_RNA', group.by='seurat_clusters', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_sub$nFeature_SCT), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
figure <- ggarrange(p1, p3, p5, p2, p4, p6, ncol = 3, nrow = 2,
                    common.legend = TRUE,legend="none")
# figure

In [96]:
# read in marker genes list from a file
markers.fp2 <- '/nfs/lab/projects/mouse_pdac/references/mouse_pdac_CAF_markers.txt'
marker.genes <- scan(markers.fp2,what="",sep="\n")

# p1 prints out a Dot Plot showing the strength of marker genes for each seurat cluster
options(repr.plot.width=20, repr.plot.height=5)
p1 <- DotPlot(adata_sub, assay='SCT', features=marker.genes, cluster.idents=TRUE) 
p1 <- p1 + theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
# p1

## Identifying Clusters

In [88]:
### Make a list of cell type assignments, one per cluster (aka cluster 1 = beta_1, cluster 2 = alpha_1, etc)
major_celltypes = c('Neutrophils','Ductal cells','T and NK cells', 'Ductal cells','Ductal cells/EMT-like cells',
                    'Fibroblast', 'B cells','B cells','Macrophage/Neutrophils','Ductal cells',
                    'B cells','B cells', 'Endothelial/Perivascular','Acinar cells','Dendritic cells') 

num_clusters <- length(unique(adata_sub$seurat_clusters))
celltype_key = seq(1:num_clusters)
names(celltype_key) = major_celltypes

### Map the cell type assignments onto the barcodes vector using the cluster #s 
assigned_celltypes1 = as.vector(mapvalues(adata_sub[[]]$seurat_clusters, from=seq(1:num_clusters), 
         to = names(celltype_key), warn_missing = TRUE))

names(assigned_celltypes1) = Cells(adata_sub)

### Add cell types into Seurat object metadata
adata_sub$assigned_celltype = assigned_celltypes1

In [89]:
# This makes a UMAP of the intermediate step with initial celltype annotations before subclustering 
options(repr.plot.width=18, repr.plot.height=6)
p1 <- DimPlot(adata_sub, reduction='umap.rna', group.by='assigned_celltype', label=TRUE, label.size=6, repel=TRUE) + ggtitle('RNA')
p1 <- p1 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('RNA only')
p2 <- DimPlot(adata_sub, reduction='umap.atac', group.by='assigned_celltype', label=TRUE, label.size=6, repel=TRUE) + ggtitle('ATAC')
p2 <- p2 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC only')
p3 <- DimPlot(adata_sub, reduction='umap.wnn', group.by='assigned_celltype', label=TRUE, label.size=6, repel=TRUE) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('Combined')
# p1 + p2 + p3 & NoLegend() & theme(plot.title=element_text(hjust=0.5))

# Subcluster

In [46]:
### Map the cell type assignments onto the barcodes vector using the cluster #s 
num_clusters <- length(unique(adata_sub$seurat_clusters))

assigned_celltypes1 = as.vector(mapvalues(adata_sub[[]]$seurat_clusters, from=seq(1:num_clusters), 
         to = names(celltype_key), warn_missing = TRUE))

names(assigned_celltypes1) = Cells(adata_sub)

### Add cell types into Seurat object metadata
adata_sub$assigned_celltypes1 = assigned_celltypes1

In [47]:
### make new list of cell type assignments -- run this ONCE
assigned_celltypes1 <- copy(assigned_celltypes1)

In [98]:
### Subcluster -- run for each cluster you want to subcluster
# set the cluster number you want to subcluster
sc.num <- 6

#find sub clusters and make a subset object
adata_sub <- FindSubCluster(adata_sub, cluster=sc.num, algorithm=4, resolution=.5, graph.name='wsnn')
adata.subset <- subset(adata_sub, subset=seurat_clusters==sc.num)

#print the # cells in each subcluster
cluster_samples <- adata_sub[[]][adata_sub[[]]$seurat_clusters==sc.num,]
table(cluster_samples$sub.cluster)

#plot UMAP of subclusters
options(repr.plot.width=6, repr.plot.height=6)
p1 <- DimPlot(adata.subset, reduction='umap.wnn', group.by='sub.cluster', label=TRUE, label.size=6, repel=TRUE)
# p1

#plot metrics boxplots
options(repr.plot.width=24, repr.plot.height=16)
p1 <- VlnPlot(adata_sub, features='nCount_SCT', group.by='sub.cluster', log=TRUE, pt.size=0) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(adata_sub$nCount_SCT), linetype='dashed') + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p2 <- VlnPlot(adata_sub, features='nFeature_SCT', group.by='sub.cluster', log=TRUE, pt.size=0) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(adata_sub$nFeature_SCT), linetype='dashed') + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p3 <- VlnPlot(adata_sub, features='nCount_ATAC', group.by='sub.cluster', log=TRUE, pt.size=0) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(adata_sub$nCount_ATAC), linetype='dashed') + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p4 <- VlnPlot(adata_sub, features='nFeature_ATAC', group.by='sub.cluster', log=TRUE, pt.size=0) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(adata_sub$nFeature_ATAC), linetype='dashed') + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p5 <- VlnPlot(adata_sub, features='nCount_RNA', group.by='sub.cluster', log=TRUE, pt.size=0) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(adata_sub$nCount_SCT), linetype='dashed') + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
p6 <- VlnPlot(adata_sub, features='nFeature_RNA', group.by='sub.cluster', log=TRUE, pt.size=0) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(adata_sub$nFeature_SCT), linetype='dashed') + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
figure <- ggarrange(p1, p3, p5, p2, p4, p6, ncol = 3, nrow = 2,
                    common.legend = TRUE,legend="none")
# figure

#plot the sample distribution in subclusters
options(repr.plot.width=25, repr.plot.height=5)
adata_sub$value <- 1
p2 <- ggplot(adata_sub[[]], aes(fill=library, y=value, x=sub.cluster)) + 
geom_bar(position=position_fill(reverse=TRUE), stat='identity') + xlab('') + 
ylab('Percentage') + NoLegend() +
theme(plot.title=element_text(hjust=0.5,size=25),
      axis.text=element_text(size=20),axis.ticks=element_blank())
# p2

# plot marker gene dotplot (replace marker.genes with your list)
markers.fp2 <- '/nfs/lab/projects/mouse_pdac/references/mouse_pdac_markers_short.txt'
marker.genes <- scan(markers.fp2,what="",sep="\n")
options(repr.plot.width=25, repr.plot.height=6)
p1 <- DotPlot(adata.subset, assay='SCT', group.by='sub.cluster', features=marker.genes, cluster.idents=TRUE) 
p1 <- p1 + theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
# p1

1 singletons identified. 5 final clusters.




6_1 6_2 6_3 6_4 6_5 
233 152 146  99  42 

In [143]:
#reassign celltypes for subclusters (if marker genes separate better) -- must do this for your cluster of interest before any subcluster another cluster
assigned_celltypes1[row.names(adata[[]])[adata[[]]$sub.cluster == '1_1']] = 'Myeloid' #example
assigned_celltypes1[row.names(adata[[]])[adata[[]]$sub.cluster == '1_2']] = 'Myeloid' 
assigned_celltypes1[row.names(adata[[]])[adata[[]]$sub.cluster == '1_3']] = 'Myeloid' 

In [48]:
### After subclustering everything - run once
adata_sub$Fibroblasts <- assigned_celltypes1

## Assigning celltypes to clusters

In [52]:
#setting color palette
celltypes <- unique(adata_sub$Fibroblasts)

celltype_colors <- c('Acinar cells'='#251351', 'B cells'='#3B52A5', 'Dendritic cells'='#5AB1BB', 
                     'Ductal cells'='#618040', 'EMT-like cells'='#AFE079', 'Endothelial cells'='#FF773D',
                     'Macrophages'='#CAA9DE', 'Neutrophils'='#D84727',
                     'Perivascular cells'='#EDA2C0', 'T and NK cells'='#E3C15D', 'myCAF'='#710CAB',
                     'apCAF'='#4FF0E0', 'iCAF'='#C2C0C0')

In [91]:
# Prints out final UMAP with finalized celltypes and assigned colors
options(repr.plot.width=18, repr.plot.height=6)
p1 <- DimPlot(adata_sub, reduction='umap.rna', group.by='Fibroblasts', label=FALSE, label.size=6, repel=TRUE, cols=celltype_colors) + ggtitle('RNA')
p1 <- p1 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('RNA only') + theme (legend.position="none")
p2 <- DimPlot(adata_sub, reduction='umap.atac', group.by='Fibroblasts', label=FALSE, label.size=6, repel=TRUE, cols=celltype_colors) + ggtitle('ATAC')
p2 <- p2 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC only') + theme(legend.position="none")
p3 <- DimPlot(adata_sub, reduction='umap.wnn', group.by='Fibroblasts', label=FALSE, label.size=6, repel=TRUE, cols=celltype_colors) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('Combined')
# p1 + p2 + p3 & theme(plot.title=element_text(hjust=0.5))

In [92]:
 # Ordered marker gene dot plot for bulk celltypes
ordered_celltypes <- names(celltype_colors)
marker.genes2 <- c('Il7r', # T and NK cells
                   'Flt1', # Perivascular cells
                   'Csf1r', # Myeloid
                   'Srgn', # Neutrophils
                   'Arg1', # Macrophage
                   'Col5a1', # Fibroblasts
                   'Egfl7', # Endothelial cells
                   'Hmga2', # EMT-like cells
                   'Prom1', # Ductal cells
                   'Cacnb3', # Dendritic cells
                   'Bank1', # B cells
                   'Ctrb1') # Acinar cells)
                   

# p1 prints out a Dot Plot of the ordered marker genes for bulk celltypes
options(repr.plot.width=12, repr.plot.height=7)
p1 <- DotPlot(adata_sub, assay='SCT', features=marker.genes2, 
              cluster.idents=TRUE, group.by = 'Fibroblasts') 
p1 <- p1 + theme(axis.text.x=element_text(angle=45, hjust=1, size=16),
                 axis.text.y=element_text(size=16),
                 legend.text=element_text(size=14),
                 legend.title=element_text(size=16)) + xlab('') + ylab('')
p1$data$id <- factor(x = p1$data$id, levels=rev(ordered_celltypes))
# p1

# CAF subpop plots

In [60]:
adata_caf <- subset(adata_sub, subset = assigned_celltypes1 == 'Fibroblasts')

In [93]:
# Ordered marker gene dot plot for CAF subpopulations
marker.genes2 <- c('Svep1', # iCAF
                   'Plpp3',
                   'Scara5',
                   'Ptgis', # apCAF
                   'Fth1',
                   'Cxadr',
                   'Col12a1', # myCAF
                   'Tnc',
                   'Col8a1')

# p1 prints out a Dot Plot of the ordered marker genes for CAF subpops
options(repr.plot.width=8, repr.plot.height=4)
p1 <- DotPlot(adata_caf, assay='SCT', features=marker.genes2, 
              cluster.idents=TRUE, group.by = 'Fibroblasts') 
p1 <- p1 + theme(axis.text.x=element_text(angle=45, hjust=1, size=16),
                 axis.text.y=element_text(size=16),
                 legend.text=element_text(size=14),
                 legend.title=element_text(size=16)) + xlab('') + ylab('')
p1$data$id <- factor(x = p1$data$id, levels=rev(ordered_celltypes))
# p1

In [99]:
# QC Violin Plots -- figure prints out a violin plot showing different features
options(repr.plot.width=32, repr.plot.height=8)
p1 <- VlnPlot(adata_caf, features='nFeature_RNA', group.by='Fibroblasts', pt.size=0, col=celltype_colors, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_caf$nFeature_SCT), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20), axis.title.x=element_blank())
p2 <- VlnPlot(adata_caf, features='nCount_ATAC', group.by='Fibroblasts', pt.size=0, col=celltype_colors, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_caf$nCount_ATAC), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20), axis.title.x=element_blank())
p3 <- VlnPlot(adata_caf, features='percent.mt', group.by='Fibroblasts', pt.size=0, col=celltype_colors, log=FALSE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_caf$percent.mt), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20), axis.title.x=element_blank())
p4 <- VlnPlot(adata_caf, features='atac_mitochondrial_reads', group.by='Fibroblasts', pt.size=0, col=celltype_colors, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(adata_caf$atac_mitochondrial_reads), linetype='dashed', lw=2) + 
  theme(plot.title = element_text(size = 25), axis.text = element_text(size=20), axis.title.x=element_blank())
figure <- ggarrange(p1, p2, p3, p4, ncol = 4, nrow = 1,
                    common.legend = TRUE,legend="none")
# figure

In [100]:
# Proportion of celltype for each sample
options(repr.plot.width=8, repr.plot.height=4)
theme_set(
    theme_classic())
p2 <- ggplot(adata_caf[[]], aes(fill=Fibroblasts, y=value, x=Genotype)) + 
        geom_bar(position=position_fill(reverse=TRUE), stat='identity') + 
        scale_fill_manual(values=celltype_colors) + scale_y_continuous(limits=c(0,1)) +
        ylab('Cell Type Composition')  + labs(fill="Celltypes") +
        theme(plot.title=element_text(hjust=0.5,size=25),
              axis.title=element_text(size=20),
              axis.text.y=element_text(size=15, hjust=1.25),
              axis.text.x=element_text(size=15, hjust=1.25),
              axis.ticks=element_blank())
p2$data$Fibroblasts <- factor(x = p2$data$Fibroblasts, levels = celltypes)
# p2 # A bar plot showing the proportion of celltype composition for caf subpops categorized by genotype

# Add in disease info and make more plots!

In [72]:
classifications <- c('AKPC','KPC')
class_key <- str_split_fixed(samples, '_h', 2)[,1]
names(class_key) <- classifications

#add classification status into object metadat
classification = as.vector(mapvalues(adata_sub[[]]$library, from=class_key, 
                                     to = names(class_key), warn_missing = TRUE))

# Add cell types into Seurat object metadata
adata_sub$classification = classification

The following `from` values were not present in `x`: KA_41_1_2_KA_39_1_2, KA_42_1_2_KA_40_1_2



In [73]:
class_colors <- c('AKPC'='#6DB5CF','KPC'='#7FCE6E')

In [101]:
# UMAP colored by classification
options(repr.plot.width=7.5, repr.plot.height=6)
p1 <- DimPlot(adata_caf, reduction='umap.wnn', group.by='Genotype', label=F, label.size=6, 
              repel=TRUE, raster=FALSE, cols=class_colors, pt.size=0.01)
p1 <- p1 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('Combined') +
      theme(legend.text=element_text(size=18),legend.title=element_text(size=18),
            plot.title=element_blank(), axis.title=element_blank(),
            axis.text=element_blank(),axis.ticks=element_blank(),
            axis.line=element_blank())
# p1 # a UMAP of the CAF subpops colored by classification of genotype

In [102]:
# Bar Plot colored by classification
options(repr.plot.width=12, repr.plot.height=6)
theme_set(theme_classic())
p1 <- ggplot(adata_sub[[]], aes(fill=Genotype, y=value, x=Fibroblasts)) + 
        geom_bar(position=position_fill(reverse=TRUE), stat='identity') + xlab('') + 
        ylab('Sample Classification') + scale_fill_manual(values=class_colors) + 
        theme(plot.title=element_text(hjust=0.5,size=25),
              axis.title=element_text(size=20),
              axis.text.y=element_text(size=15, hjust=1.25),
              axis.text.x=element_text(size=20, angle=45, hjust=0.95),
              axis.ticks=element_blank(),
              legend.text=element_text(size=18), 
              legend.title=element_blank())
# p1 # A bar plot of proportion of genotype classification for each celltype

# Save Seurat object to an RDS

In [77]:
# Change file path here
outdir <- "/nfs/lab/ylee/mouse_pdac"
rds_fp2 <- file.path(outdir,'230718_2sample_merged_with_celltypes_and_CAFs.rds')
saveRDS(adata_sub, file = rds_fp2)

In [103]:
# sessionInfo() # print to see sessionInfo of this notebook